In [1]:
import json
import csv

import pandas as pd

# Read the JSON data from the 'movies.json' file
with open('../data/zheng/movies.json', 'r') as json_file:
    movie_data = json.load(json_file)




In [2]:
df=pd.read_json('../data/zheng/movies.json')
# Sample JSON data
json_line = '{"Title":"The Land Girls","US_Gross":146083,"Worldwide_Gross":146083,"US_DVD_Sales":null,"Production_Budget":8000000,"Release_Date":"12-Jun-98","MPAA_Rating":"R","Running_Time_min":null,"Distributor":"Gramercy","Source":null,"Major_Genre":null,"Creative_Type":null,"Director":null,"Rotten_Tomatoes_Rating":null,"IMDB_Rating":6.1,"IMDB_Votes":1071}'

In [3]:
df.head()

,Title,US Gross,Worldwide Gross,US DVD Sales,Production Budget,Release Date,MPAA Rating,Running Time min,Distributor,Source,Major Genre,Creative Type,Director,Rotten Tomatoes Rating,IMDB Rating,IMDB Votes
0,The Land Girls,146083.0,146083.0,NaN,8000000.0,Jun 12 1998,R,NaN,Gramercy,None,None,None,None,NaN,6.1,1071.0
1,"First Love, Last Rites",10876.0,10876.0,NaN,300000.0,Aug 07 1998,R,NaN,Strand,None,Drama,None,None,NaN,6.9,207.0
2,I Married a Strange Person,203134.0,203134.0,NaN,250000.0,Aug 28 1998,None,NaN,Lionsgate,None,Comedy,None,None,NaN,6.8,865.0
3,Let's Talk About Sex,373615.0,373615.0,NaN,300000.0,Sep 11 1998,None,NaN,Fine Line,None,Comedy,None,None,13.0,NaN,NaN
4,Slam,1009819.0,1087521.0,NaN,1000000.0,Oct 09 1998,R,NaN,Trimark,Original Screenplay,Drama,Contemporary Fiction,None,62.0,3.4,165.0


In [4]:
df.to_csv("../data/zheng/movies_underlying_data.csv", index=True)

In [5]:
marks=['point','tick','bar','circle']
x_attributes=['Title', 'US_Gross', 'Worldwide_Gross', 'US_DVD_Sales', 'Production_Budget', 'Release_Date', 'MPAA_Rating', 'Running_Time_min', 'Distributor', 'Source', 'Major_Genre', 'Creative_Type', 'Director', 'Rotten_Tomatoes_Rating', 'IMDB_Rating', 'IMDB_Votes']
y_attributes=['Title', 'US_Gross', 'Worldwide_Gross', 'US_DVD_Sales', 'Production_Budget', 'Release_Date', 'MPAA_Rating', 'Running_Time_min', 'Distributor', 'Source', 'Major_Genre', 'Creative_Type', 'Director', 'Rotten_Tomatoes_Rating', 'IMDB_Rating', 'IMDB_Votes']
## Open a CSV file for writing
with open('../data/zheng/combinations.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(['id', 'mark', 'x_attribute', 'y_attribute'])

    # Initialize an ID counter
    id_counter = 0

    # Write all combinations
    for mark in marks:
        for x_attr in x_attributes:
            for y_attr in y_attributes:
                writer.writerow([id_counter, mark, x_attr, y_attr])
                id_counter += 1

print("CSV file 'combinations.csv' has been created.")

CSV file 'combinations.csv' has been created.


In [44]:
def get_interaction_id(search_entry):
        # Open the CSV file and search for the entry
    with open('../data/zheng/combinations.csv', 'r') as csv_file:
        reader = csv.reader(csv_file)

        # Skip the header row
        next(reader)

        # Initialize a variable to store the found ID
        found_id = 1155

        # Iterate through the rows
        for row in reader:
            row_values = row[1:]  # Exclude the 'id' column
            if row_values == search_entry:
                found_id = row[0]
                break

    return found_id

In [45]:
def search_attribute(row):

    fieldnames = ['Title', 'US_Gross', 'Worldwide_Gross', 'US_DVD_Sales', 'Production_Budget', 'Release_Date', 'MPAA_Rating', 'Running_Time_min', 'Distributor', 'Source', 'Major_Genre', 'Creative_Type', 'Director', 'Rotten_Tomatoes_Rating', 'IMDB_Rating', 'IMDB_Votes']
    matches=[]
    for i in fieldnames:
        if i in row and len(matches)<2:
            matches.append(i)
    if len(matches)==0:
        matches.append(None)
        matches.append(None)
    elif len(matches)==1:
        matches.append(None)

    return matches

In [47]:
import os
import re
user_interactions_path='../data/zheng/processed_csv/'
csv_files=os.listdir(user_interactions_path)

fieldnames = ['Title', 'US_Gross', 'Worldwide_Gross', 'US_DVD_Sales', 'Production_Budget', 'Release_Date', 'MPAA_Rating', 'Running_Time_min', 'Distributor', 'Source', 'Major_Genre', 'Creative_Type', 'Director', 'Rotten_Tomatoes_Rating', 'IMDB_Rating', 'IMDB_Votes']

combined_user_interactions=pd.DataFrame(columns=['user','interaction_session'])
for csv_filename in csv_files:
    print("Converting '{}'...".format(csv_filename))
    user_name=csv_filename.split('.')[0]
    df = pd.read_csv(os.path.join(user_interactions_path, csv_filename))
    interactions=[]
    print('Total size of interaction log',len(df))
    for index, row in df.iterrows():
        value=row['Value']
       # Use regular expressions to extract mark, x_attribute, and y_attribute values
        if len(str(value)) < 15 or row['Interaction'] in ('scroll up', 'scroll down','typed in answer'):
            matches= [None,None,None]
        else:
            mark_match = re.search(r'mark:(\w+);', value)
            # x_attr_match = re.search(r'encoding:-quantitative-x<aggregate>([\w<>-]+),', value)
            # y_attr_match = re.search(r'encoding:-quantitative-y<aggregate>([\w<>-]+),', value)
            mark = mark_match.group(1) if mark_match else None
            # x_attribute = x_attr_match.group(1) if x_attr_match else None
            # y_attribute = y_attr_match.group(1) if y_attr_match else None
            matches= search_attribute(value)
            matches = [mark] + matches

        if None in matches:
            continue
        else:
            match_to_dataset=int(get_interaction_id(matches))
            #print(match_to_dataset)
            interactions.append(match_to_dataset)
    print(user_name,len(interactions))
    combined_user_interactions=combined_user_interactions.append({'user':user_name,'interaction_session':interactions},ignore_index=True)
    interactions=[]
print("Conversion complete.")

Converting 'stu31_ade_p3_logs.csv'...
Total size of interaction log 1170
stu31_ade_p3_logs 192
Converting 'pro13_ace_p3_logs.csv'...
Total size of interaction log 1199
pro13_ace_p3_logs 427
Converting 'stu15_ade_p3_logs.csv'...
Total size of interaction log 3762
stu15_ade_p3_logs 2134
Converting 'stu29_ade_p4_logs.csv'...
Total size of interaction log 2351
stu29_ade_p4_logs 1316
Converting 'pro9_acf_p3_logs.csv'...
Total size of interaction log 2026
pro9_acf_p3_logs 566
Converting 'stu26_ace_p4_logs.csv'...
Total size of interaction log 5315
stu26_ace_p4_logs 3255
Converting 'pro8_acf_p4_logs.csv'...
Total size of interaction log 999
pro8_acf_p4_logs 182
Converting 'pro20_ade_p4_logs.csv'...
Total size of interaction log 3491
pro20_ade_p4_logs 676
Converting 'stu30_adf_p3_logs.csv'...
Total size of interaction log 1140
stu30_adf_p3_logs 247
Converting 'pro21_ade_p3_logs.csv'...
Total size of interaction log 3072
pro21_ade_p3_logs 899
Converting 'stu37_adf_p3_logs.csv'...
Total size of 

In [41]:
combined_user_interactions

,user,interaction_session
0,stu31_ade_p3_logs,"[334, 45, 334, 331, 334, 331, 331, 331, 331, 3..."
1,pro13_ace_p3_logs,"[45, 45, 45, 45, 45, 40, 40, 40, 40, 40, 40, 4..."
2,stu15_ade_p3_logs,"[335, 335, 335, 335, 335, 328, 328, 328, 328, ..."
3,stu29_ade_p4_logs,"[672, 672, 672, 672, 345, 345, 345, 345, 345, ..."
4,pro9_acf_p3_logs,"[44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 4..."
...,...,...
67,pro25_acf_p3_logs,"[44, 44, 44, 44, 44, 44, 44, 39, 44, 42, 42, 4..."
68,stu4_ade_p4_logs,"[674, 674, 674, 674, 674, 674, 674, 674, 113, ..."
69,pro18_adf_p3_logs,"[691, 691, 691, 691, 669, 669, 669, 669, 669, ..."
70,pro24_adf_p4_logs,"[671, 671, 671, 671, 671, 671, 671, 671, 671, ..."


In [42]:
combined_user_interactions.to_csv("../data/zheng/competing_movies_interactions.csv", index=False)

In [43]:
combined_user_interactions['interaction_session'].count()

72